# models

> All code related to our models 

In [ ]:
#| default_exp models

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
from torch import nn
import torch.nn.functional as F
from fastai.data.all import *

In [ ]:
#| export
class stagerNetAAE(nn.Module):
    def __init__(self, channels: int=23, timestamps: int=3001,
                    acc_factor: int=8, dropout_rate: float=0.5, level: int=0,
                    latent_dim: int=128, gan_depth: int=3, k_pool_size: int=13,
                    recons_weight: float=0.1, adv_weight: float=0.5, classif_weight: float=0.4
                ):
        super(stagerNetAAE, self).__init__()
        
        self.channels = channels #number of input channels (spatial)
        self.timestamps = timestamps #number of input timestamps (temporal)
        self.acc_factor = acc_factor
        self.latent_dim = latent_dim #embed_dim
        self.k_pool_size = k_pool_size #embed_dim
        self.dropout_rate = dropout_rate
        self.gan_depth = gan_depth
        self.gen_train = True
        self.level = level
        self.global_loss = False
        self.discrim_loss = 1.0
        self.adv_loss = 1.0
        self.recons_weight = recons_weight
        self.adv_weight = adv_weight
        self.classif_weight = classif_weight
        
        #=============Encoder=============#
        self.conv1 = nn.Conv2d(1, self.channels, (1, self.channels), stride=(1, 1))
        self.conv2 = nn.Conv2d(1, 16, (self.timestamps//60,1), stride=(1,1))
        self.conv3 = nn.Conv2d(16, 16, (self.timestamps//60,1), stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(self.k_pool_size,1), return_indices=True) 
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.bn_lin = nn.BatchNorm1d(num_features=self.latent_dim)

        self.fc_input_size = self._get_fc_input_size(self.timestamps)
        self.fc_z = nn.Linear(self.fc_input_size, self.latent_dim)

        #=============Decoder=============#
        self.decoder_input = nn.Linear(self.latent_dim, self.fc_input_size)
        self.unpool = nn.MaxUnpool2d(kernel_size = (self.k_pool_size,1))
        self.deconv1 = nn.ConvTranspose2d(self.channels, 1, (1, self.channels), stride=(1, 1))
        self.deconv2 = nn.ConvTranspose2d(16, 1, (self.timestamps//60,1), stride=(1,1))
        self.deconv3 = nn.ConvTranspose2d(16, 16, (self.timestamps//60,1), stride=(1,1))

        #===============GAN===============#
        fcs = ['fc_crit0','fc_crit1','fc_crit2','fc_crit3','fc_crit4']
        bns = ['bn_crit0','bn_crit1','bn_crit2','bn_crit3','bn_crit4']
        for i in range(self.gan_depth-1):
            self.add_module(fcs[i], nn.Linear(self.latent_dim//2**(i), self.latent_dim//2**(i+1)))
            self.add_module(bns[i], nn.BatchNorm1d(num_features=self.latent_dim//2**(i+1)))
        self.add_module(fcs[self.gan_depth-1], nn.Linear(self.latent_dim//2**(self.gan_depth-1), 1))
        
        #============Classifier============#
        self.fc_clf_discr1 = nn.Linear(self.latent_dim, 2)
            
    def _get_fc_input_size(self, timestamps):
        x = torch.randn(16, 1, timestamps, self.channels)
        x,_,_,_,_ = self._forward_conv(x)
        return x.shape[1]

    def _forward_conv(self, inp):
        inp = self.conv1(inp)
        inp = inp.permute(0, 3, 2, 1)
        inp = self.conv2(inp)
        input_mp1 = inp.detach().clone()
        inp, ind_maxpool1 = self.pool(inp)
        inp = F.relu(inp)
        inp = self.batchnorm1(inp)
        inp = self.conv3(inp)
        input_mp2 = inp.detach().clone()
        inp, ind_maxpool2 = self.pool(inp)
        inp = F.relu(inp)
        result = self.batchnorm2(inp)
        result = torch.flatten(result, start_dim=1)
        result = F.dropout(result, p=self.dropout_rate)
        return result, ind_maxpool1, ind_maxpool2, input_mp1, input_mp2

    def encode(self, inp: Tensor) -> List[Tensor]:
        result, ind_maxpool1, ind_maxpool2, input_mp1, input_mp2 = self._forward_conv(inp)

        z = self.fc_z(result)
        zi = F.relu(self.bn_lin(z))

        return [zi, ind_maxpool1, ind_maxpool2, input_mp1, input_mp2]
    
    def decode(self, z: Tensor, ind1, ind2, in1, in2) -> Tensor:
        x = self.decoder_input(z)        
        x = x.view(self.current_bs, 16, -1, self.channels) 
        x = self.unpool(x, indices=ind2, output_size = in2.size())
        x = F.relu(x)
        x = self.batchnorm2(x)
        x = self.deconv3(x)
        x = self.unpool(x, indices=ind1, output_size = in1.size())
        x = F.relu(x)
        x = self.batchnorm1(x)
        x = self.deconv2(x)
        x = x.permute(0, 3, 2, 1)
        result = self.deconv1(x)        
        return result

    def latent_gan(self, zi: Tensor) -> Tensor:
        x = zi.view(-1,self.latent_dim)
        for i in range(self.gan_depth-1):
            x = getattr(self,f'fc_crit{i}')(x)
            x = F.leaky_relu(getattr(self,f'bn_crit{i}')(x),negative_slope=0.2)
        x = getattr(self,f'fc_crit{self.gan_depth-1}')(x)
        x = F.sigmoid(x)
        return x

    def forward(self, inp: Tensor, **kwargs) -> Tensor:
        if inp.dim() < 4:
            inp = inp.unsqueeze(1)
        self.ae_input = inp # needed to compute reconstruction loss

        x = inp.permute(0, 1, 3, 2)
        self.current_bs = x.shape[0]
        zi, ind1, ind2, in1, in2 = self.encode(x)

        self.zi = zi # needed to further display the latent space

        zi_gan = zi.view(-1, self.latent_dim, 1)
        self.gan_fake = self.latent_gan(zi_gan)
        z = torch.randn_like(zi_gan)
        z = F.dropout(z, p=self.dropout_rate)
        self.gan_real = self.latent_gan(z)

        decoded = self.decode(zi, ind1, ind2, in1, in2)
        self.decoded = decoded
        self.decoded = decoded.permute(0, 1, 3, 2)

        self.pred_class = F.softmax(self.fc_clf_discr1(zi)).to(dev)
  
        pred_eeg = self.pred_class

        return pred_eeg

    def ae_loss_func(self, output, target):
        delta = .5
        huber = nn.HuberLoss(delta=delta)
        recons_loss = (huber(self.decoded, self.ae_input) +
                2*huber(self.decoded.std(dim=-1), self.ae_input.std(dim=-1)) # avoid the decoded signal to stay at 0
                )
        return recons_loss


    def aae_loss_func_monitosed(self, output, target):
        delta = .5
        huber = nn.HuberLoss(delta=delta)
        adversarial_loss = nn.BCELoss()

        self.recons_loss = (huber(self.decoded, self.ae_input) +
            2*huber(self.decoded.std(dim=-1), self.ae_input.std(dim=-1)) # avoid the decoded signal to stay at 0
            )

        if self.gen_train: #generator loss
            # Measures generator's ability to fool the discriminator
            discrim_tmp = torch.tensor(self.discrim_loss, requires_grad=False)
            valid = torch.ones_like(self.gan_fake, requires_grad=False).detach()
            self.adv_loss = adversarial_loss(self.gan_fake, valid) + discrim_tmp
            loss = self.adv_weight * self.adv_loss + self.recons_weight * self.recons_loss # + 0.5 * self.classif_loss

        else: #discriminator loss
            # Measure discriminator's ability to classify real from generated samples
            valid = 0.99*torch.ones_like(self.gan_real, requires_grad=False).detach()
            valid = F.dropout(valid, p=self.dropout_rate)
            fake = 0.01*torch.ones_like(self.gan_fake, requires_grad=False).detach()
            fake = F.dropout(fake, p=self.dropout_rate)
            self.real_loss = adversarial_loss(self.gan_real, valid)
            self.fake_loss = adversarial_loss(self.gan_fake, fake)
            self.discrim_loss = 0.6 * self.real_loss + 0.4 * self.fake_loss
            loss = 10 * self.discrim_loss

        return loss

    def aae_classif_loss_func_monitosed(self, output, target):
        delta = .5
        huber = nn.HuberLoss(delta=delta)
        adversarial_loss = nn.BCELoss()
        bce = nn.CrossEntropyLoss()
        rnd_weight = torch.empty(self.gan_fake.shape, requires_grad=False).uniform_(0.9, 1.0).to('cuda:0').detach()

        self.classif_loss = bce(output, target)

        self.recons_loss = (huber(self.decoded, self.ae_input) +
            2*huber(self.decoded.std(dim=-1), self.ae_input.std(dim=-1)) # avoid the decoded signal to stay at 0
            )

        if self.gen_train: #generator loss
            # Measures generator's ability to fool the discriminator
            valid = rnd_weight * torch.ones_like(self.gan_fake, requires_grad=False).detach()
            discrim_tmp = torch.tensor(self.discrim_loss, requires_grad=False)
            self.adv_loss = (adversarial_loss(self.gan_fake.mean(0), valid.mean(0))
                            + adversarial_loss(self.gan_fake.std(0), valid.std(0))
                            + discrim_tmp)
            loss = (self.adv_weight * self.adv_loss + self.recons_weight * self.recons_loss 
                    + self.classif_weight * self.classif_loss)

        else: #discriminator loss
            # Measure discriminator's ability to classify real from generated samples
            valid = rnd_weight * torch.ones_like(self.gan_real, requires_grad=False).detach()
            fake = (1-rnd_weight) * torch.ones_like(self.gan_fake, requires_grad=False).detach()
            self.real_loss = (adversarial_loss(self.gan_real.mean(0), valid.mean(0))
                            + adversarial_loss(self.gan_real.std(0), valid.std(0)))
            self.fake_loss = (adversarial_loss(self.gan_fake.mean(0), fake.mean(0))
                            + adversarial_loss(self.gan_fake.std(0), fake.std(0)))
            self.discrim_loss = 0.6 * self.real_loss + 0.4 * self.fake_loss
            loss = 10 * self.discrim_loss #the x10 factor avoid the discriminator loss to be responsible for early stopping

        return loss

In [ ]:
show_doc(stagerNetAAE)

---

### stagerNetAAE

>      stagerNetAAE (channels:int=23, timestamps:int=3001, acc_factor:int=8,
>                    dropout_rate:float=0.5, level:int=0, latent_dim:int=128,
>                    gan_depth:int=3, k_pool_size:int=13,
>                    recons_weight:float=0.1, adv_weight:float=0.5,
>                    classif_weight:float=0.4)

Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this module is in training or
                evaluation mode.
:vartype training: bool